In [1]:
#%matplotlib notebook
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from tqdm import tqdm_notebook
import emcee
import corner

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [2]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def parallel_process(array, function, n_jobs=None, use_kwargs=False, front_num=0):
    """
        A parallel version of the map function with a progress bar. 

        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
            front_num (int, default=3): The number of iterations to run serially before kicking off the 
                parallel job. This can be useful for catching bugs
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    #We run the first few iterations serially to catch bugs
    if front_num > 0:
        front = [function(**a) if use_kwargs else function(a) for a in array[:front_num]]
    #If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        [function(**a) if use_kwargs else function(a) for a in tqdm_notebook(array[front_num:])]
        return 
    #Assemble the workers
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        #Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[front_num:]]
        else:
            futures = [pool.submit(function, a) for a in array[front_num:]]
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': False,
            'leave': True
        }
        #Print out the progress as tasks complete
        for f in tqdm_notebook(as_completed(futures), **kwargs):
            pass

In [3]:
def beta_model(So, r, rc, beta):
    ''' Beta model with 3 parameters. 
    So -- normalization
    rc -- core radius
    beta -- powerlaw slope

    '''
    
    return So * ( 1.0 + (r / rc)**2)**(-3.0 * beta + 0.5)

In [4]:
def chi2(model, y, y_err):
    '''Chi error. We are going to square this to make it the chi2 error.'''
    return np.sum(((model - y) / y_err)**2)

In [5]:
def like(theta, x, y, yerr):
    So, rc, beta, bg = theta
    model = beta_model(So, x, rc, beta) + bg
    #return -0.5 * np.sum(np.log(2 * np.pi * yerr) + (y - model)**2 / yerr)
    return -chi2(model, y, yerr)

def prior(theta):
    So, rc, beta, bg = theta
    if So < 0:
        return -np.inf
    elif rc < 0 or rc > 10:
        return -np.inf
    elif beta < 0 or beta > 3: 
        return -np.inf
    elif bg < 0 or bg > 1:
        return -np.inf
    else:
        return 0.0

def prob(theta, x, y, yerr):
    lp = prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + like(theta, x, y, yerr)

In [6]:
def fit_profile(name, outpath):
    
    # swift pixelscale in degrees
    pixsc =  6.548089E-04 * 3600
    
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        srcs = f'{outpath}/{name}/{name}_vtp.detect'
        # now we need to read the individual detections
        detects = Table.read(srcs, hdu=1)
    else:
        return
    
    # we are going to fit profiles to the top 3 biggest 
    detects.sort(['SRC_AREA'], reverse=True)
    
    # open a file to write out the results
    outfile = open(f'{outpath}/{name}/{name}_mcmcfits.txt', 'w')    
    outfile.write('# Field ID So_50 So_84 So_16 rc_50 rc_84 rc_16 beta_50 beta_84 beta_16 bg_50 bg_84 bg_16\n')
    
    # loop over the sources -- only the first 3
    for i in detects['INDEX'][:3]:
        
        if os.path.isfile(f'{outpath}/{name}/{name}_vtp_{i}.radprof'):
            data = Table.read(f'{outpath}/{name}/{name}_vtp_{i}.radprof', format='ascii', header_start=2)
        else:
            continue
            
        # don't try to fit things if there isn't any data
        if len(data) <= 60:
            continue
        
        # x-axis, in arcminutes
        x = (data['r1'] + data['r2'])/ 2. / 60. * pixsc
        
        # this is the parameter fitting
        ndim = 4  # number of parameters in the model
        nwalkers = 100  # number of MCMC walkers
        nburn = 100  # "burn-in" period to let chains stabilize
        nsteps = 500  # number of MCMC steps to take

        pos = [np.array([0.001, 1., 1., 0.001]) + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
        sampler = emcee.EnsembleSampler(nwalkers, ndim, prob, args=(x, data['sb'], data['sb_err']))
        sampler.run_mcmc(pos, nsteps)
        emcee_trace = sampler.get_chain(discard=nburn, thin=15, flat=True)
        
        # plot the result
        fignum = np.random.randint(1, 1000)
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6.5, 5), num=fignum)
        # data
        ax.errorbar(x, data['sb'], yerr=data['sb_err'], fmt='o', label='data')
        for So, rc, beta, bg in emcee_trace[np.random.randint(len(emcee_trace), size=100)]:
            ax.plot(x, beta_model(So, x, rc, beta) + bg, color='k', alpha=0.1)
        ylims = ax.get_ylim()
        
        #fit -- median (and error) values
        So, rc, beta, bg = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(emcee_trace, [16, 50, 84], axis=0)))
        ax.plot(x, beta_model(So[0], x, rc[0], beta[0]) + bg[0], label=r'$\beta$ + bg')
        ax.plot(x, beta_model(So[0], x, rc[0], beta[0]), label=r'$\beta$')
        
        # write out the fit parameters
        outfile.write(f'{name} ')
        outfile.write(f'{i} ')
        outfile.write(f'{So[0]:.5f} {So[1]:.5f} {So[2]:.5f} ')
        outfile.write(f'{rc[0]:.5f} {rc[1]:.5f} {rc[2]:.5f} ')
        outfile.write(f'{beta[0]:.5f} {beta[1]:.5f} {beta[2]:.5f} ')
        outfile.write(f'{bg[0]:.5f} {bg[1]:.5f} {bg[2]:.5f}\n')

        # rc and bg lines
        ax.axhline(bg[0], label='bg', zorder=0, lw=1)
        ax.axvline(rc[0], zorder=0, lw=1)
        
        ax.semilogy()

        if ylims[0] < 1e-5:
            ax.text(rc[0] + 0.1, 2e-5, f"rc = {rc[0]:.2f}'", rotation='vertical', ha='left')
            ax.set_ylim(1e-5, ylims[1])
        else:
            ax.text(rc[0] + 0.1, ylims[0], f"rc = {rc[0]:.2f}'", rotation='vertical', ha='left')
            ax.set_ylim(ylims)
        ax.set_xlabel('Radius [arcmin]')
        ax.set_ylabel('Flux [cnts/s]')
        ax.legend(loc='upper right')
        
        fig.savefig(f'{outpath}/{name}/{name}_radproffit_{i}.png', bbox='tight', dpi=180)
        plt.close(fig)
        
        # add a corner plot of the fits.
        # plot the bg and So in log to make them fit on the plot better
        emcee_trace[:,0] = np.log(emcee_trace[:,0])
        emcee_trace[:,3] = np.log(emcee_trace[:,3])
        f = corner.corner(emcee_trace, labels=['log So', 'rc', r'$\beta$', 'log bg'],
                    bins=[50, 50, 50, 50],
                    smooth=True,
                    fill_contours=True)

        f.savefig(f'{outpath}/{name}/{name}_corner_{i}.png', bbox='tight')
        plt.close(f)
    outfile.close()

In [7]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath} for n in data['NAME']]
parallel_process(arr, fit_profile, use_kwargs=True, n_jobs=8)

In [7]:
#d = Table.read('./data_full/PSZ1_G083.35+76.41/PSZ1_G083.35+76.41_vtp_7.radprof', format='ascii', header_start=2)
# d = Table.read('./data_full/PSZ1_G057.42-10.77/PSZ1_G057.42-10.77_vtp_2.radprof', format='ascii', header_start=2)
# d = Table.read('./data_full/PSZ2_G356.21-43.11/PSZ2_G356.21-43.11_vtp_0.radprof', format='ascii', header_start=2)
#d = Table.read('./data_full/PSZ2_G003.21-76.04/PSZ2_G003.21-76.04_vtp_1.radprof', format='ascii', header_start=2)
 d = Table.read('./data_full/PSZ2_G305.76+44.79/PSZ2_G305.76+44.79_vtp_1.radprof', format='ascii', header_start=2)

#PSZ2_G028.08+10.79


In [8]:
# swift pixelscale in degrees
pixsc =  6.548089E-04 * 3600
x = (d['r1'] + d['r2'])/ 2. / 60. * pixsc

In [9]:
# this is the parameter fitting
ndim = 4  # number of parameters in the model                                  
nwalkers = 100  # number of MCMC walkers                                        
nburn = 100  # "burn-in" period to let chains stabilize                        
nsteps = 500  # number of MCMC steps to take  

pos = [np.array([0.001, 1., 1., 0.001]) + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, prob, args=(x, d['sb'], d['sb_err']))
sampler.run_mcmc(pos, nsteps)
samples = sampler.get_chain(discard=nburn, thin=15, flat=True)

In [ ]:

So, rc, beta, bg = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))
print(So)
print(rc)
print(beta)
print(bg)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6.5, 5))
# data
ax.errorbar(x, d['sb'], yerr=d['sb_err'], fmt='o', label='data')
for So, rc, beta, bg in samples[np.random.randint(len(samples), size=100)]:
    ax.plot(x, beta_model(So, x, rc, beta) + bg, color='k', alpha=0.1)
ylims = ax.get_ylim()

#fit
# So, rc, beta, bg = result['x']


So, rc, beta, bg = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))

ax.plot(x, beta_model(So[0], x, rc[0], beta[0]) + bg[0], label=r'$\beta$ + bg')
ax.plot(x, beta_model(So[0], x, rc[0], beta[0]), label=r'$\beta$')


ax.axhline(bg[0], label='bg', zorder=0, lw=1)

ax.text(rc[0] + 0.1, 2e-5, f"rc = {rc[0]:.2f}'", rotation='vertical', ha='left')
ax.axvline(rc[0], zorder=0, lw=1)
ax.semilogy()

if ylims[0] < 1e-5:
    ax.set_ylim(1e-5, ylims[1])
else:
    ax.set_ylim(ylims)
ax.set_xlabel('Radius [arcmin]')
ax.set_ylabel('Flux [cnts/s]')
ax.legend(loc='upper right')
#fig.savefig(f'/home/boada/PSZ1_G057.42-10.77_radproffit_2.png', bbox='tight', dpi=180)

In [ ]:
samples[:,3] = np.log(samples[:,3])
samples[:,0] = np.log(samples[:,0])

In [ ]:
f = corner.corner(samples, labels=['log So', 'rc', r'$\beta$', 'log bg'],
                    bins=[50, 50, 50, 50],
                    smooth=True,
                    fill_contours=True)

In [ ]:
fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ['So', 'rc', 'beta', 'bg']
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");